# ft_pedidos

Esse notebook é responsável por reunir todas as informações úteis de cada pedido do e-commerce


## Configurações Iniciais

### Configurações Spark

In [0]:
spark.conf.set("spark.sql.ansi.enabled", "false")

### Importando as Bibliotecas


In [0]:
from pyspark.sql import functions as F

### Carregando os Dados

In [0]:
df_orders = (
    spark.read.csv(
        "/Volumes/workspace/default/e_commerce/olist_orders_dataset.csv",
        header=True,
        inferSchema=True
    )
)

df_order_item = (
    spark.read.csv(
        "/Volumes/workspace/default/e_commerce/olist_order_items_dataset.csv",
        header=True,
        inferSchema=True
    )
    .withColumnRenamed("price", "net_price")
)

df_order_payments = (
    spark.read.csv(
        "/Volumes/workspace/default/e_commerce/olist_order_payments_dataset.csv",
        header=True,
        inferSchema=True
    )
    .withColumn(
        "payment_type",
        F.when(
            F.col("payment_type") == "not_defined",
            "NÃO DEFINIDO"
        )
        .when(
            F.col("payment_type") == "credit_card",
            "CARTÃO DE CRÉDITO"
        )
        .when(
            F.col("payment_type") == "debit_card",
            "CARTÃO DE DÉBITO"
        )
        .otherwise(F.upper(F.col("payment_type")))
    )
    .withColumn(
        "installment_value",
        F.col("payment_value") / F.col("payment_installments")
    )
)

df_order_reviews = (
    spark.read.csv(
        "/Volumes/workspace/default/e_commerce/olist_order_reviews_dataset.csv",
        header=True,
        inferSchema=True
    )
    .drop("review_creation_date", "review_answer_timestamp")
    .dropna(subset=["order_id"])
)

## Transformação dos Dados

In [0]:
df_ft_pedidos = (
    df_orders
    .join(
        df_order_item,
        on="order_id",
        how="left"
    )
    .join(
        df_order_payments,
        on="order_id",
        how="left"
    )
    .join(
        df_order_reviews,
        on="order_id",
        how="left"
    )
    .withColumn(
        "delivery_time_seconds",
        (F.col("order_delivered_customer_date").cast("long") - F.col("order_purchase_timestamp").cast("long")).cast("int")
    )
    .withColumn(
        "delivery_time_hours",
        F.col("delivery_time_seconds") / 3600
    )
    .withColumn(
        "delivery_time_days",
        F.col("delivery_time_seconds") / (3600 * 24)
    )
    .withColumn(
        "freight_percent",
        F.col("freight_value") / F.col("payment_value")
    )
    .withColumn(
        "review_comment_message",
        F.regexp_replace(F.col("review_comment_message"), '"', "'")
    )
)

## Salvamento dos Dados

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ifrs_dev_ecommerce.gold.ft_pedidos (
    order_id STRING COMMENT "ID do pedido",
    customer_id STRING COMMENT "ID do cliente",
    order_status STRING COMMENT "Status do pedido",
    order_purchase_timestamp TIMESTAMP COMMENT "Data/hora da compra",
    order_approved_at TIMESTAMP COMMENT "Data/hora de aprovação",
    order_delivered_carrier_date TIMESTAMP COMMENT "Data/hora de envio para transportadora",
    order_delivered_customer_date TIMESTAMP COMMENT "Data/hora de entrega ao cliente",
    order_estimated_delivery_date TIMESTAMP COMMENT "Data estimada de entrega",
    order_item_id INT COMMENT "ID do item do pedido",
    product_id STRING COMMENT "ID do produto",
    seller_id STRING COMMENT "ID do vendedor",
    shipping_limit_date TIMESTAMP COMMENT "Data limite de envio",
    net_price DOUBLE COMMENT "Preço líquido do item",
    freight_value DOUBLE COMMENT "Valor do frete",
    payment_sequential INT COMMENT "Sequencial do pagamento",
    payment_type STRING COMMENT "Tipo de pagamento",
    payment_installments INT COMMENT "Número de parcelas",
    payment_value DOUBLE COMMENT "Valor do pagamento",
    installment_value DOUBLE COMMENT "Valor da parcela",
    review_id STRING COMMENT "ID da avaliação",
    review_score STRING COMMENT "Nota da avaliação",
    review_comment_title STRING COMMENT "Título do comentário da avaliação",
    review_comment_message STRING COMMENT "Mensagem do comentário da avaliação",
    delivery_time_seconds INT COMMENT "Tempo de entrega em segundos",
    delivery_time_hours DOUBLE COMMENT "Tempo de entrega em horas",
    delivery_time_days DOUBLE COMMENT "Tempo de entrega em dias",
    freight_percent DOUBLE COMMENT "Percentual do frete sobre o valor pago",
    CONSTRAINT fk_customer_id FOREIGN KEY (customer_id) REFERENCES ifrs_dev_ecommerce.gold.dim_clientes (customer_id),
    CONSTRAINT fk_product_id FOREIGN KEY (product_id) REFERENCES ifrs_dev_ecommerce.gold.dim_produtos (product_id),
    CONSTRAINT fk_seller_id FOREIGN KEY (seller_id) REFERENCES ifrs_dev_ecommerce.gold.dim_fornecedores (seller_id)
)
USING DELTA
CLUSTER BY (order_id, customer_id)
TBLPROPERTIES ("quality"="gold", "layer"="data-engineering")
COMMENT "Fato de pedidos do e-commerce contendo informações detalhadas de cada pedido.";

-- Adicionando tags à tabela
ALTER TABLE ifrs_dev_ecommerce.gold.ft_pedidos SET TAGS ("quality"="gold", "layer"="data-engineering");

In [0]:
(
    df_ft_pedidos
    .write.mode("overwrite")
    .format("delta")
    .saveAsTable("ifrs_dev_ecommerce.gold.ft_pedidos")
)